# Codenames - Sprint 5

<font color='green'><b>Renee, Husein, George

<font color='blue'>Libraries

In [1]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import random
import PySimpleGUI as sg
from gensim.models import KeyedVectors
import gensim
import gensim.downloader as api
import pickle
from scipy.spatial.distance import cdist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import pickle
import string
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
from spellchecker import SpellChecker
import pygame

pygame 2.4.0 (SDL 2.26.4, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


[nltk_data] Downloading package wordnet to /Users/reneec/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/reneec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<font color='blue'>Read CSV File

In [2]:
df = pd.read_csv('nouns.csv')
df.head()

,0,people
0,1,history
1,2,way
2,3,art
3,4,world
4,5,information


<font color='blue'>DF to List

In [3]:
df = df['people']
word_corpus = df.tolist()
word_corpus.remove('a')
print(word_corpus)

['history', 'way', 'art', 'world', 'information', 'map', 'two', 'family', 'government', 'health', 'system', 'computer', 'meat', 'year', 'thanks', 'music', 'person', 'reading', 'method', 'data', 'food', 'understanding', 'theory', 'law', 'bird', 'literature', 'problem', 'software', 'control', 'knowledge', 'power', 'ability', 'economics', 'love', 'internet', 'television', 'science', 'library', 'nature', 'fact', 'product', 'idea', 'temperature', 'investment', 'area', 'society', 'activity', 'story', 'industry', 'media', 'thing', 'oven', 'community', 'definition', 'safety', 'quality', 'development', 'language', 'management', 'player', 'variety', 'video', 'week', 'security', 'country', 'exam', 'movie', 'organization', 'equipment', 'physics', 'analysis', 'policy', 'series', 'thought', 'basis', 'boyfriend', 'direction', 'strategy', 'technology', 'army', 'camera', 'freedom', 'paper', 'environment', 'child', 'instance', 'month', 'truth', 'marketing', 'university', 'writing', 'article', 'departmen

In [4]:
# Get the English stop words
stop_words = set(stopwords.words('english'))

# Remove the stop words from the list
words = [word for word in word_corpus if word.lower() not in stop_words]

print(words)

['history', 'way', 'art', 'world', 'information', 'map', 'two', 'family', 'government', 'health', 'system', 'computer', 'meat', 'year', 'thanks', 'music', 'person', 'reading', 'method', 'data', 'food', 'understanding', 'theory', 'law', 'bird', 'literature', 'problem', 'software', 'control', 'knowledge', 'power', 'ability', 'economics', 'love', 'internet', 'television', 'science', 'library', 'nature', 'fact', 'product', 'idea', 'temperature', 'investment', 'area', 'society', 'activity', 'story', 'industry', 'media', 'thing', 'oven', 'community', 'definition', 'safety', 'quality', 'development', 'language', 'management', 'player', 'variety', 'video', 'week', 'security', 'country', 'exam', 'movie', 'organization', 'equipment', 'physics', 'analysis', 'policy', 'series', 'thought', 'basis', 'boyfriend', 'direction', 'strategy', 'technology', 'army', 'camera', 'freedom', 'paper', 'environment', 'child', 'instance', 'month', 'truth', 'marketing', 'university', 'writing', 'article', 'departmen

<font color='blue'>Team Association

In [5]:
random_words = list(words)
random.shuffle(random_words)
blue_team = random.sample(random_words, 8)
red_team = random.sample(set(random_words) - set(blue_team), 8)
civilians = random.sample(set(random_words) - set(blue_team) - set(red_team), 8)
assassin = random.sample(set(random_words) - set(blue_team) - set(red_team) - set(civilians), 1)

print(blue_team)
print(red_team)
print(civilians)
print(assassin)

['battle', 'restaurant', 'slip', 'charge', 'city', 'love', 'connection', 'drag']
['poetry', 'raise', 'organization', 'champion', 'scheme', 'quarter', 'period', 'cover']
['refuse', 'savings', 'bread', 'affect', 'comment', 'assistance', 'math', 'post']
['understanding']


/var/folders/h_/j7jfxsn929n19yn9z525bp9h0000gn/T/ipykernel_21939/329972402.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  red_team = random.sample(set(random_words) - set(blue_team), 8)
/var/folders/h_/j7jfxsn929n19yn9z525bp9h0000gn/T/ipykernel_21939/329972402.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  civilians = random.sample(set(random_words) - set(blue_team) - set(red_team), 8)
/var/folders/h_/j7jfxsn929n19yn9z525bp9h0000gn/T/ipykernel_21939/329972402.py:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  assassin = random.sample(set(random_words) - set(blue_team) - set(red_team) - set(civilians), 1)


<font color='blue'>Codenames for Matrix

In [6]:
codenames = blue_team + red_team + civilians + assassin
random.shuffle(codenames)
print(codenames)

['period', 'affect', 'comment', 'city', 'organization', 'restaurant', 'post', 'assistance', 'cover', 'quarter', 'raise', 'connection', 'savings', 'drag', 'love', 'battle', 'champion', 'slip', 'refuse', 'scheme', 'bread', 'understanding', 'charge', 'math', 'poetry']


<font color='blue'>Copy Codenames and Blue team words

In [7]:
codenames1 = codenames.copy()
blue_team1 = blue_team.copy()

In [8]:
# Initialize the mixer module
pygame.mixer.init()

In [9]:
# Load the sound effects
click_sound = pygame.mixer.Sound('button.wav')
score_sound = pygame.mixer.Sound('score.wav')
end_game_sound = pygame.mixer.Sound('assassin.wav')

<font color='blue'>Load the glove dictionary from the binary file

In [10]:
with open('glove.pkl', 'rb') as f:
    glove = pickle.load(f)

<font color='blue'>Initialize the WordNetLemmatizer

In [11]:
wnl = WordNetLemmatizer()

<font color='blue'>Define a function to check if a word is a plural form of any of the input words

In [12]:
def is_plural(word):
    singular = wnl.lemmatize(word, pos=wordnet.NOUN)
    return singular != word

In [13]:
def has_punctuation(word):
    return any(char in string.punctuation for char in word)

In [14]:
def find_most_similar_and_order(user_clue, codenames, glove, topn=5):
    # Calculate the cosine similarity between the input word and each word in the list
    similarity_scores = []
    for word in codenames:
        if word in glove:
            similarity_score = np.dot(glove[user_clue], glove[word]) / (np.linalg.norm(glove[user_clue]) * np.linalg.norm(glove[word]))
            similarity_scores.append(similarity_score)
        else:
            similarity_scores.append(-1.0) # assign a default similarity score for out-of-vocabulary words

    # Order the word list based on the cosine similarity scores
    ordered_word_list = [word for _, word in sorted(zip(similarity_scores, codenames), reverse=True)]

    # Find the most similar words to the input word
    if user_clue not in glove:
        most_similar_words = []
    else:
        user_clue_vector = glove[user_clue]
        distances = cdist(user_clue_vector.reshape(1,-1), list(glove.values()), 'cosine')[0]
        sorted_distances = np.argsort(distances)
        closest_words = [list(glove.keys())[i] for i in sorted_distances[:topn+1]]
        most_similar_words = [w for w in closest_words if w != user_clue and not is_plural(w) and not w.endswith('ing') and not w.endswith('ed') and not has_punctuation(w)][:topn]

    return ordered_word_list, most_similar_words

In [15]:
similar_words = {word: find_most_similar_and_order(word, codenames, glove)[1] for word in codenames}
print(similar_words)

{'period': ['early'], 'affect': ['adversely', 'depend'], 'comment': ['statement'], 'city': ['town', 'downtown', 'mayor', 'area'], 'organization': ['organisation', 'nonprofit'], 'restaurant': ['cafe', 'eatery', 'hotel'], 'post': ['office', 'editorial'], 'assistance': ['aid', 'provide', 'assist', 'relief'], 'cover': [], 'quarter': ['earnings', 'fourth', 'half', 'profit'], 'raise': ['increase', 'interest'], 'connection': ['link', 'involvement'], 'savings': ['pension', 'insurance', 'deposit'], 'drag': ['rupaul', 'aerodynamic', 'friction'], 'love': ['passion', 'romantic'], 'battle': ['fought', 'fight', 'war'], 'champion': ['champ', 'medalist'], 'slip': ['catch', 'caught', 'slide'], 'refuse': ['accept', 'refusal'], 'scheme': ['scam', 'plan', 'fraud', 'ponzi'], 'bread': ['loaf', 'flour', 'butter', 'cheese'], 'understanding': ['knowledge', 'understand', 'cooperation', 'respect', 'profound'], 'charge': ['guilty'], 'math': ['mathematics', 'literacy', 'proficiency'], 'poetry': ['literature', 'pro

<font color='blue'>Game State - Score

In [16]:
game_state = {
    'blue_score': 0,
    'red_score': 0,
}

<font color='blue'>Set Theme

In [17]:
sg.theme('DarkPurple')

'DarkPurple'

In [18]:
spell = SpellChecker()

<font color='blue'>Define Layout with User and Computer Clues

In [19]:
user_clue = sg.Text('', key="-USER_CLUE-", font=('Trebuchet MS', 18))
print_clue = sg.Text('', key="-COMPUTER_CLUE-", font=('Trebuchet MS', 18))
num_words_str = sg.Text('', key="-NUM_WORDS-", font=('Trebuchet MS', 18))

# Define the layout for the start window
start_layout = [
    [sg.Text('Welcome to the Game!', font=('Trebuchet MS', 16))],
    [sg.Button('Start Game', font=('Trebuchet MS', 16), key='-START_GAME-')],
    [sg.Button('Exit Game', font=('Trebuchet MS', 16), key='-EXIT_GAME-')]
]

<font color='blue'>Window

In [20]:
# Create the start window
start_window = sg.Window('Codenames', start_layout, 
                         element_justification='center',
                         size = (800,700))

<font color='blue'>Clue Generator

In [21]:
def generate_clue(blue_team_words):
    target_word = random.choice(blue_team_words)
    most_similar_word = random.choice(similar_words[target_word])
    clue = f"{most_similar_word.title()}"
    num_words = sum([1 for word in blue_team_words if most_similar_word in similar_words[word]])
    return clue, num_words

blue_team_words = blue_team.copy()
clue = generate_clue(blue_team_words)

<font color='blue'>User Clue Function - similarity scores

<font color='blue'>Define Custom Colors

In [22]:
# Defining Custom tile colors
custom_color_red = ('#6c757d', '#d00000')
custom_color_blue = ('#6c757d', 'blue')
custom_color_civilians = ('#6c757d','#D3D3D3')
custom_color_assassin = ('#6c757d', '#000000')

<font color='blue'>Button Color Change Function

In [23]:
def change_color(button):
    word = button.get_text()
    if word in blue_team:
        button.update(button_color = custom_color_blue)
        game_state['blue_score'] += 1
        game_window['blue_score'].update(game_state['blue_score'])
        score_sound.play()
        if game_state['blue_score'] == 8:
            end_game("blue")
    elif word in red_team:
        button.update(button_color = custom_color_red)
        game_state['red_score'] += 1
        game_window['red_score'].update(game_state['red_score'])
        if game_state['red_score'] == 8:
            end_game("red")
    elif word in civilians:
        button.update(button_color = custom_color_civilians)
    else:
        button.update(button_color = custom_color_assassin)
        # End the game if the assassin word is clicked
        for i in range(5):
            for j in range(5):
                button = game_window[(i, j)]
                button.update(disabled=True)
        end_game_sound.play()
        
# Function to check if score has reached point 8 for either of the teams and end the game
def end_game(winner):
    # Disable all buttons and display the winner
    for i in range(5):
        for j in range(5):
            button = game_window[(i, j)]
            button.update(disabled=True)
    game_window['-COMPUTER_CLUE-'].update(f"{winner.upper()} TEAM WINS!")
    end_game_sound.play()

<font color='blue'>Codemaster Layout(2)

In [24]:
codenames2 = codenames.copy()
layout2 = []
for i in range(5):
    row2 = []
    for j in range(5):
        word = codenames2.pop(0)
        if word in blue_team:
            button_color=('blue', 'blue')
        elif word in red_team:
            button_color=('red', 'red')
        elif word in civilians:
            button_color=('gray', 'gray')
        else:
            button_color=('black', 'black')
        row2.append(sg.Button(key=(i, j), button_color = button_color, size=(2, 2)))
    layout2.append(row2)

<font color='blue'>Window 2

In [25]:
window2 = sg.Window('Codemaster Matrix', layout2, size = (400,400), finalize=True)

<font color='blue'>Run Codenames Game

In [26]:
# Event loop for the Start_window
while True:
    start_event, start_values = start_window.read()
    if start_event in (sg.WIN_CLOSED, '-EXIT_GAME-'):
        break
    elif start_event == '-START_GAME-':
        # Closing start window after 'Start Game' is clicked
        start_window.close()
        
        # Define the layout for the actual game_window
        left = [
            [sg.Text('User Clue: ', font=('Trebuchet MS', 16)), sg.InputText(key="-USER_CLUE_INPUT-", size = (20,2))],
            [sg.Text('Enter Number: ', font=('Trebuchet MS', 16)), sg.InputText(key="-NUM_WORDS_INPUT-", size = (20,2))],
            [user_clue],
            [num_words_str],
            [sg.Column([[sg.Button('Submit Clue', font=('Trebuchet MS', 16))]])]   
            ]

        center = [
            [sg.Text('WE ARE ON THE BLUE TEAM', font=('Trebuchet MS', 16), text_color='white')],
            [sg.Text('Blue Score:', font=('Trebuchet MS', 16)), sg.Text(game_state['blue_score'], key='blue_score', font=('Trebuchet MS', 16))],
            [sg.Text('Red Score:', font=('Trebuchet MS', 16)), sg.Text(game_state['red_score'], key='red_score', font=('Trebuchet MS', 16))]
            ]

        right = [
            [sg.Text('Computer Clue: ', font=('Trebuchet MS', 16))],
            [print_clue],
            [sg.Column([[sg.Button('Next Clue', font=('Trebuchet MS', 16))]], pad=(0, 20))]
            ]
        
        # Adding the game_layout
        layout = [
            [sg.Column(left, element_justification='left'),
             sg.Column(center, element_justification='center'),
             sg.Column(right, element_justification='right')],
            ]

        for i in range(5):
            row = []
            for j in range(5):
                word = codenames1.pop(0)
                row.append(sg.Button(word, key=(i, j), size=(12, 2), font=('Trebuchet MS', 12)))
            layout.append(row)
        
        # Creating the game_window
        game_window = sg.Window('Codenames Matrix', layout, size = (800,700))
        event2, values2 = window2.read(timeout=100)
        if event2 == (sg.WINDOW_CLOSED, sg.TIMEOUT_EVENT):
            break
        
        while True:
            game_event, game_values = game_window.read()
            if game_event in (sg.WIN_CLOSED, '-EXIT_GAME-'):
                break
            # Event Loop for the game window - to handle event 'submit clue'
            elif game_event == 'Submit Clue':
                user_clue = values["-USER_CLUE_INPUT-"]
                # Check if the user's input word is spelled correctly
                misspelled = spell.unknown([user_clue])
                if misspelled:
                    # Get the most likely correction
                    correction = spell.correction(user_clue)
                    # Replace the user's input word with the correction
                    user_clue = correction
                game_window["-USER_CLUE-"].update(user_clue)
                num_words_str = values["-NUM_WORDS_INPUT-"]
                game_window["-NUM_WORDS-"].update(num_words_str)
                num_words = int(num_words_str)  # convert the value to an integer
                comp_guess, most_similar_words = find_most_similar_and_order(user_clue, codenames, glove)
                for i in range(5):
                    for j in range(5):
                        button = game_window[(i, j)]
                        for word in comp_guess[:num_words]:
                            if word == button.get_text():
                                change_color(button)
                                break 
                            
            # Event Loop to handle event 'Next Clue'
            elif game_event == 'Next Clue':
                clue, num_words = generate_clue(blue_team_words)
                print_clue.update(clue)
                game_window["-NUM_WORDS-"].update(num_words)
            elif isinstance(game_event, tuple):               # Check if a button was clicked
                button = game_window[game_event]
                change_color(button)
                word = button.get_text()
                if word in blue_team:
                    blue_team_words.remove(word)
                click_sound.play()

<font color='blue'>Close Windows

In [27]:
start_window.close()
window2.close()